<a href="https://colab.research.google.com/github/vipin-jangra/face-age-estimation-CNN/blob/main/D2_CNN2_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import keras
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.applications import ResNet101
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.applications.resnet50 import preprocess_input

In [ ]:
def preprocess_image(image_path):
    # Read grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize image to (224, 224)
    image = cv2.resize(image, (224, 224))
    # Convert grayscale to RGB by stacking the single channel
    image_rgb = np.stack((image,) * 3, axis=-1)
    # Preprocess image for ResNet50 model
    image_rgb = preprocess_input(image_rgb)
    return image_rgb

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!unzip '/content/drive/MyDrive/Dataset/fg-net.zip'

Mounted at /content/drive


In [ ]:
# Define age ranges
age_ranges = [(1, 2), (3, 9), (10, 20), (21, 27), (28, 45), (46, 65), (66, 116)]

# Function to map age to range index
def age_to_range(age, age_ranges):
    for idx, (start, end) in enumerate(age_ranges):
        if start <= age <= end:
            return idx
    return None

In [ ]:
def preprocess_labels(labels, num_classes):
    # One-hot encode labels
    labels = to_categorical(labels, num_classes=num_classes)
    return labels

In [ ]:
# Load and preprocess the dataset
image_dir = '/content/FGNET'
images=[]
image_paths = []
age_labels = []
age_groups = []
target_size = (224, 224)

for filename in os.listdir(image_dir):
  if filename.endswith(".jpg"):
    age = int(filename.split('A')[1])
    range_index = age_to_range(age,age_ranges)
    age_labels.append(range_index)

    image_path = os.path.join(image_dir, filename)
    image = preprocess_image(image_path)
    images.append(image)
    image_paths.append(image_path)

images = np.array(images)
age_groups = age_labels
age_labels = preprocess_labels(age_labels, len(age_ranges))

In [ ]:
print(f'number of images : {len(images)}')

number of images : 9931


In [ ]:
import pandas as pd
df = pd.DataFrame()
age_intervals = [age_to_range(age,age_ranges) for age in age_groups]
df['images_path'],df['age'] = image_paths,age_intervals
df.head(10)

NameError: name 'age_groups' is not defined

In [ ]:
plt.figure(figsize=(20,20))
samples = df.iloc[0:16]

for index,sample,age in samples.itertuples():
  plt.subplot(4,4,index+1)
  img = load_img(sample)
  img = np.array(img)
  plt.axis('off')
  plt.title(f'Age:{age}')
  plt.imshow(img)

In [ ]:
#age distribution
sns.displot(data=age_labels,kde=True)

In [ ]:
train_images, test_images, train_age_groups, test_age_groups = train_test_split(images, age_labels, test_size=0.2, random_state=42)


In [ ]:
# Load the pre-trained ResNet50 model
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(132, activation='relu')(x)
predictions = Dense(len(age_ranges), activation='softmax')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Dataset/D3_CNN2_A3.h5',  # Filepath to save the best model
    monitor='val_accuracy',  # Metric to monitor
    verbose=1,
    save_best_only=True,  # Only save the best model
    mode='max'  # Mode to determine best (maximizing validation accuracy)
)


In [ ]:
model.fit(train_images, train_age_groups, validation_split=0.2, epochs=60, batch_size=32, callbacks=[model_checkpoint], shuffle=False)


Epoch 1/60
199/199 [==============================] - ETA: 0s - loss: 1.0779 - accuracy: 0.5695
Epoch 1: val_accuracy improved from -inf to 0.56199, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


199/199 [==============================] - 58s 103ms/step - loss: 1.0779 - accuracy: 0.5695 - val_loss: 1.1683 - val_accuracy: 0.5620
Epoch 2/60
199/199 [==============================] - ETA: 0s - loss: 0.4084 - accuracy: 0.8535
Epoch 2: val_accuracy improved from 0.56199 to 0.60604, saving model to best_model.h5
199/199 [==============================] - 15s 76ms/step - loss: 0.4084 - accuracy: 0.8535 - val_loss: 1.1887 - val_accuracy: 0.6060
Epoch 3/60
199/199 [==============================] - ETA: 0s - loss: 0.1292 - accuracy: 0.9651
Epoch 3: val_accuracy did not improve from 0.60604
199/199 [==============================] - 14s 70ms/step - loss: 0.1292 - accuracy: 0.9651 - val_loss: 1.8848 - val_accuracy: 0.5400
Epoch 4/60
199/199 [==============================] - ETA: 0s - loss: 0.1203 - accuracy: 0.9663
Epoch 4: val_accuracy did not improve from 0.60604
199/199 [==============================] - 14s 70ms/step - loss: 0.1203 - accuracy: 0.9663 - val_loss: 1.5488 - val_accuracy

In [ ]:
# Load the best model
model.load_weights('/content/drive/MyDrive/Dataset/D3_CNN2_A3.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_age_groups)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


63/63 [==============================] - 2s 31ms/step - loss: 2.1241 - accuracy: 0.6452
Test Accuracy: 64.52%


In [ ]:

predictions = model.predict(test_images)

# Convert predictions to class labels if needed
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_age_groups, axis=1)

63/63 [==============================] - 2s 23ms/step


In [ ]:
# Example metrics calculation
from sklearn.metrics import classification_report

print(classification_report(true_classes, predicted_classes))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88       379
           1       0.63      0.62      0.62       240
           2       0.62      0.55      0.58       277
           3       0.48      0.65      0.55       234
           4       0.56      0.52      0.54       339
           5       0.61      0.61      0.61       340
           6       0.72      0.63      0.67       178

    accuracy                           0.65      1987
   macro avg       0.64      0.64      0.64      1987
weighted avg       0.65      0.65      0.65      1987



In [ ]:
# Calculate classification metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.65
Precision: 0.65
Recall: 0.65
F1-score: 0.65


In [ ]:
# Calculate accuracy for each class
class_accuracies = {}
for idx, (start, end) in enumerate(age_ranges):
    # Filter predictions and true labels for the current class
    mask = (true_classes == idx)
    accuracy = accuracy_score(true_classes[mask], predicted_classes[mask])
    class_accuracies[f'{start}-{end}'] = accuracy

In [ ]:
# Print accuracy for each class
for age_range, accuracy in class_accuracies.items():
    print(f'Accuracy for age range {age_range}: {accuracy:.2f}')

Accuracy for age range 1-2: 0.88
Accuracy for age range 3-9: 0.62
Accuracy for age range 10-20: 0.55
Accuracy for age range 21-27: 0.65
Accuracy for age range 28-45: 0.52
Accuracy for age range 46-65: 0.61
Accuracy for age range 66-116: 0.63


In [ ]:
from sklearn.metrics import confusion_matrix
# Detailed classification report
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=[f'{start}-{end}' for start, end in age_ranges]))

# Confusion Matrix
print('\nConfusion Matrix:')
print(confusion_matrix(true_classes, predicted_classes))


Classification Report:
              precision    recall  f1-score   support

         1-2       0.89      0.88      0.88       379
         3-9       0.63      0.62      0.62       240
       10-20       0.62      0.55      0.58       277
       21-27       0.48      0.65      0.55       234
       28-45       0.56      0.52      0.54       339
       46-65       0.61      0.61      0.61       340
      66-116       0.72      0.63      0.67       178

    accuracy                           0.65      1987
   macro avg       0.64      0.64      0.64      1987
weighted avg       0.65      0.65      0.65      1987


Confusion Matrix:
[[333  38   2   3   2   1   0]
 [ 36 149  39   9   3   1   3]
 [  0  37 151  70  15   1   3]
 [  3   5  18 153  50   5   0]
 [  2   1  13  72 175  74   2]
 [  0   6  14  11  64 209  36]
 [  0   2   6   0   5  53 112]]
